In [1]:
from __future__ import print_function
import os
import argparse
import torch
import torch.backends.cudnn as cudnn
import numpy as np
# from data import cfg_mnet, cfg_re50
from layers.functions.prior_box import PriorBox
from utils.nms.py_cpu_nms import py_cpu_nms
import cv2
from models.retinaface import RetinaFace
from utils.box_utils import decode, decode_landm
from utils.timer import Timer
import matplotlib.pyplot as plt

In [2]:
FIX_AFTER_MODEL = False

In [3]:
def check_keys(model, pretrained_state_dict):
    ckpt_keys = set(pretrained_state_dict.keys())
    model_keys = set(model.state_dict().keys())
    used_pretrained_keys = model_keys & ckpt_keys
    unused_pretrained_keys = ckpt_keys - model_keys
    missing_keys = model_keys - ckpt_keys
    print('Missing keys:{}'.format(len(missing_keys)))
    print('Unused checkpoint keys:{}'.format(len(unused_pretrained_keys)))
    print('Used keys:{}'.format(len(used_pretrained_keys)))
    assert len(used_pretrained_keys) > 0, 'load NONE from pretrained checkpoint'
    return True


def remove_prefix(state_dict, prefix):
    ''' Old style model is stored with all names of parameters sharing common prefix 'module.' '''
    print('remove prefix \'{}\''.format(prefix))
    f = lambda x: x.split(prefix, 1)[-1] if x.startswith(prefix) else x
    return {f(key): value for key, value in state_dict.items()}


def load_model(model, pretrained_path, load_to_cpu):
    print('Loading pretrained model from {}'.format(pretrained_path))
    if load_to_cpu:
        pretrained_dict = torch.load(pretrained_path, map_location=lambda storage, loc: storage)
    else:
        device = torch.cuda.current_device()
        pretrained_dict = torch.load(pretrained_path, map_location=lambda storage, loc: storage.cuda(device))
    if "state_dict" in pretrained_dict.keys():
        pretrained_dict = remove_prefix(pretrained_dict['state_dict'], 'module.')
    else:
        pretrained_dict = remove_prefix(pretrained_dict, 'module.')
    check_keys(model, pretrained_dict)
    model.load_state_dict(pretrained_dict, strict=False)
    return model

In [4]:
torch.set_grad_enabled(False)
cfg = {
    'name': 'mobilenet0.25',
    'min_sizes': [[16, 32], [64, 128], [256, 512]],
    'steps': [8, 16, 32],
    'variance': [0.1, 0.2],
    'clip': False,
    'loc_weight': 2.0,
    'gpu_train': True,
    'batch_size': 32,
    'ngpu': 1,
    'epoch': 250,
    'decay1': 190,
    'decay2': 220,
    'image_size': 640,
    'pretrain': False,
    'return_layers': {'stage1': 1, 'stage2': 2, 'stage3': 3},
    'in_channel': 32,
    'out_channel': 64
}

# net and model
net = RetinaFace(cfg=cfg, phase = 'test')
net = load_model(net, "weights/mobilenet0.25_Final.pth", True)
net.eval()
print('Finished loading model!')
print(net)
cudnn.benchmark = True
device = torch.device("cpu" if True else "cuda")
net = net.to(device)

NumANCHOR  2
Loading pretrained model from weights/mobilenet0.25_Final.pth
remove prefix 'module.'
Missing keys:0
Unused checkpoint keys:0
Used keys:300
Finished loading model!
RetinaFace(
  (body): IntermediateLayerGetter(
    (stage1): Sequential(
      (0): Sequential(
        (0): Conv2d(3, 8, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): LeakyReLU(negative_slope=0.1, inplace=True)
      )
      (1): Sequential(
        (0): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=8, bias=False)
        (1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): LeakyReLU(negative_slope=0.1, inplace=True)
        (3): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): LeakyReLU(negative_slope=0.1, 

/tmp/ipykernel_312046/1036366865.py:24: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pretrained_dict = torch.load(pretrained_path, map_location=lambda storage, loc: storage

In [5]:
def detect_faces(img_raw,vis_thres=0.9):
    if(type(img_raw)==str):
        img_raw = cv2.imread(img_raw, cv2.IMREAD_COLOR)
    img = np.float32(img_raw)
    im_height, im_width, _ = img.shape
    scale = torch.Tensor([img.shape[1], img.shape[0], img.shape[1], img.shape[0]])
    img -= (104, 117, 123)
    img = img.transpose(2, 0, 1)
    img = torch.from_numpy(img).unsqueeze(0)
    img = img.to(device)
    scale = scale.to(device)
    _t = {'forward_pass': Timer(), 'misc': Timer()}
    resize = 1

    _t['forward_pass'].tic()
    loc, conf, landms = net(img)  # forward pass
    _t['forward_pass'].toc()
    _t['misc'].tic()
    priorbox = PriorBox(cfg, image_size=(im_height, im_width))
    priors = priorbox.forward()
    priors = priors.to(device)
    prior_data = priors.data
    boxes = decode(loc.data.squeeze(0), prior_data, cfg['variance'])
    boxes = boxes * scale / resize
    boxes = boxes.cpu().numpy()
    scores = conf.squeeze(0).data.cpu().numpy()[:, 1]
    landms = decode_landm(landms.data.squeeze(0), prior_data, cfg['variance'])
    scale1 = torch.Tensor([img.shape[3], img.shape[2], img.shape[3], img.shape[2],
                            img.shape[3], img.shape[2], img.shape[3], img.shape[2],
                            img.shape[3], img.shape[2]])
    scale1 = scale1.to(device)
    landms = landms * scale1 / resize
    landms = landms.cpu().numpy()

    # ignore low scores
    inds = np.where(scores > 0.02)[0]
    boxes = boxes[inds]
    landms = landms[inds]
    scores = scores[inds]

    # keep top-K before NMS
    # order = scores.argsort()[::-1][:args.top_k]
    order = scores.argsort()[::-1]
    boxes = boxes[order]
    landms = landms[order]
    scores = scores[order]

    # do NMS
    dets = np.hstack((boxes, scores[:, np.newaxis])).astype(np.float32, copy=False)
    keep = py_cpu_nms(dets, 0.4)

    dets = dets[keep, :]
    landms = landms[keep]

    # keep top-K faster NMS
    # dets = dets[:args.keep_top_k, :]
    # landms = landms[:args.keep_top_k, :]

    dets = np.concatenate((dets, landms), axis=1)
    facess  = []
    for b in dets:
        if b[4] < vis_thres:
            continue
        xs = b[4]
        b = list(map(int, b))
        b.append(xs)
        facess.append(b)

    return facess

In [6]:
import cv2


def find_darkest_point_in_box_center(image, x, y, w, h):
    # Đọc ảnh
    # image = cv2.imread(image_path)

    # Chuyển ảnh sang thang độ xám
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Lấy kích thước ảnh
    height, width = gray_image.shape

    # Xác định tọa độ của góc trên trái và dưới phải dựa trên tâm (x, y)
    x1 = max(0, int(x - w / 2))
    y1 = max(0, int(y - h / 2))
    x2 = min(width, int(x + w / 2))
    y2 = min(height, int(y + h / 2))

    # Cắt vùng box ra khỏi ảnh
    box = gray_image[y1:y2, x1:x2]

    # Tìm tọa độ của điểm tối nhất trong box
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(box)

    # Tính lại tọa độ điểm tối nhất dựa trên ảnh gốc
    darkest_point = (min_loc[0] + x1, min_loc[1] + y1)

    return darkest_point

def draw_faces_landmarks(image_path, faces_data, output_image_path=None):
    image = cv2.imread(image_path)
    for  face_data in faces_data:
        x1, y1, x2, y2 =face_data[0:4]
        cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 2)  # Blue box
        # S = abs((x1-x2)*(y2-y1))
        # 1/4 1/8

        if(FIX_AFTER_MODEL):
            face_data[5],face_data[6] = find_darkest_point_in_box_center(image,face_data[5],face_data[6],((x2-x1)/4),((y2-y1))/8)
            face_data[7],face_data[8] = find_darkest_point_in_box_center(image,face_data[7],face_data[8],((x2-x1)/4),((y2-y1))/8)
        for item in range(5,15,2):
            cv2.circle(image, (int(face_data[item]), int(face_data[item+1])), 1, (0, 0, 255), 4)  # Red dots
        
    # Save the image with bounding boxes and landmarks
    if(output_image_path):
        cv2.imwrite(output_image_path, image)
        print(f"Image saved to {output_image_path}")
    else:
        plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        plt.axis('off')  # Tắt hiển thị trục
        plt.tight_layout()
        plt.show()

In [8]:
image_path = "c:/Users/four/Downloads/tao-dang-chup-anh-tap-the__31__80f9aa9490ee4d3f838d09a140562638.webp"
# img_raw = cv2.imread(image_path, cv2.IMREAD_COLOR)
facess = detect_faces(image_path)
print(len(facess))
print(facess)
draw_faces_landmarks(image_path,facess)

AttributeError: module 'cv2' has no attribute 'imread'

In [9]:
import math
def calculate_angle(point1, point2):
    print(point1,point2)
    # Tính độ dốc (slope)
    dy = point2[1] - point1[1]
    dx = point2[0] - point1[0]
    
    # Tránh chia cho 0
    if dx == 0:
        return 90 if dy > 0 else 270  # 90 độ nếu đi lên, 270 độ nếu đi xuống
    
    slope = dy / dx
    
    # Tính góc (tính bằng radian và chuyển sang độ)
    angle_rad = math.atan(slope)
    angle_deg = math.degrees(angle_rad)
    
    # # Điều chỉnh góc về miền từ 0 đến 360 độ
    # if angle_deg < 0:
    #     angle_deg += 360
    
    return angle_deg
from PIL import Image


def rotate_image(image_path, angle):
    # Mở ảnh
    img = Image.open(image_path)
    
    # Xoay ảnh
    rotated_img = img.rotate(angle)
    
    # Lưu ảnh đã xoay
    # rotated_img.save('rotated_image.png',quality=100)
    
    return rotated_img

# Đường dẫn tới ảnh
# image_path = 'path_to_your_image.jpg'

# Xoay ảnh 15 độ
face = facess[0]
rotated_image = rotate_image(image_path, (calculate_angle([face[5],face[6]],[face[7],face[8]])+calculate_angle([face[11],face[12]],[face[13],face[14]]))/2)

# Hiển thị ảnh đã xoay (nếu cần)
rotated_image.show()


NameError: name 'facess' is not defined

In [10]:
print(calculate_angle([face[5],face[6]],[face[7],face[8]]))

NameError: name 'face' is not defined

In [11]:
# rectangle_points = [(face[0], face[1]), (face[0]+face[2], face[1]), (face[0]+face[2], face[1]+face[3]), (face[0],face[1]+face[3])]
x1,y1,x2,y2 = face[0:4]
rectangle_points =[[x1,y1],[x2,y1],[x2,y2],[x1,y2]]
print(rectangle_points)

NameError: name 'face' is not defined

In [7]:
import math
import matplotlib.pyplot as plt

def rotate_rectangle(points, angle):
    # Tính tọa độ trung tâm
    center_x = sum(x for x, y in points) / len(points)
    center_y = sum(y for x, y in points) / len(points)

    # Chuyển đổi góc từ độ sang radian
    radian = math.radians(angle)
    
    # Ma trận xoay
    cos_angle = math.cos(radian)
    sin_angle = math.sin(radian)
    
    rotated_points = []
    
    for (x, y) in points:
        # Di chuyển điểm về gốc tọa độ
        x -= center_x
        y -= center_y
        
        # Tính tọa độ mới
        x_new = x * cos_angle - y * sin_angle
        y_new = x * sin_angle + y * cos_angle
        
        # Di chuyển điểm trở lại vị trí cũ
        x_new += center_x
        y_new += center_y
        
        rotated_points.append((x_new, y_new))
    
    return rotated_points



In [13]:
import math

angel_ = (calculate_angle([face[5],face[6]],[face[7],face[8]])+calculate_angle([face[11],face[12]],[face[13],face[14]]))/2

face = facess[0]

# Tọa độ 4 điểm hình chữ nhật (ví dụ)

x1,y1,x2,y2 = face[0:4]
rectangle_points =[[x1,y1],[x2,y1],[x2,y2],[x1,y2]]
print(rectangle_points)
# Góc xoay (độ)
angle_of_rotation = angel_

# Tính tọa độ mới sau khi xoay
new_points = rotate_rectangle(rectangle_points, angle_of_rotation)
# print()
print("Tọa độ mới sau khi xoay:", new_points)

def cut_rotated_rectangle(image_path, points):
    # Mở ảnh
    image = cv2.imread(image_path)

    # Chuyển đổi danh sách điểm thành mảng NumPy
    points = np.array(points, dtype='float32')

    # Xác định tọa độ cho hình chữ nhật mới
    width = int(max(np.linalg.norm(points[0] - points[1]), np.linalg.norm(points[2] - points[3])))
    height = int(max(np.linalg.norm(points[0] - points[3]), np.linalg.norm(points[1] - points[2])))

    # Điểm góc cho hình chữ nhật đã cắt
    dest_points = np.array([
        [0, 0],
        [width - 1, 0],
        [width - 1, height - 1],
        [0, height - 1]
    ], dtype='float32')

    # Tính ma trận biến đổi
    M = cv2.getPerspectiveTransform(points, dest_points)

    # Cắt hình
    cropped_image = cv2.warpPerspective(image, M, (width, height))

    return cropped_image


cropped_image = cut_rotated_rectangle(image_path, new_points)
plt.imshow(cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB))
# cv2.imwrite("out.jpg",cropped_image)
plt.axis(False)
plt.show()



NameError: name 'face' is not defined

In [14]:
img = cv2.imread("/home/four/Downloads/matkinh (1).png")
img.shape
print(293//2)
print(1044//4,1044//4*3)

AttributeError: module 'cv2' has no attribute 'imread'

In [15]:

# Các cấu hình cần thiết như 'cfg', 'net', và 'device' cần được định nghĩa từ trước
def overlay_glasses(frame, eye_points, glasses_img, glass_eye_points):
    # Tính kích thước của kính dựa trên khoảng cách hai mắt
    eye_dist = np.linalg.norm(eye_points[1] - eye_points[0])
    glass_width = int(1.5 * eye_dist)  # Định kích thước kính
    glass_height = int(glasses_img.shape[0] * (glass_width / glasses_img.shape[1]))
    
    # Resize kính
    resized_glasses = cv2.resize(glasses_img, (glass_width, glass_height), interpolation=cv2.INTER_AREA)
    
    # Tính vị trí đặt kính
    glasses_center = np.mean(eye_points, axis=0).astype(int)
    top_left = (glasses_center[0] - glass_width // 2, glasses_center[1] - glass_height // 2)
    
    # Overlay kính lên frame
    for i in range(glass_height):
        for j in range(glass_width):
            y, x = top_left[1] + i, top_left[0] + j
            if 0 <= x < frame.shape[1] and 0 <= y < frame.shape[0]:
                alpha = resized_glasses[i, j, 3] / 255.0  # Transparency
                for c in range(3):  # BGR channels
                    frame[y, x, c] = int(alpha * resized_glasses[i, j, c] + (1 - alpha) * frame[y, x, c])

    return frame

def detect_from_webcam():
    cap = cv2.VideoCapture(0)  # Mở webcam

    while True:
        ret, img_raw = cap.read()  # Đọc từng khung hình từ webcam
        if not ret:
            break
        data_faces = detect_faces(img_raw)
        for b in data_faces:
            cv2.rectangle(img_raw, (b[0], b[1]), (b[2], b[3]), (0, 0, 255), 2)
            cx = b[0]
            cy = b[1] + 12
            cv2.putText(img_raw, f"{(int(b[15]*10000))/10000}", (cx, cy),
                        cv2.FONT_HERSHEY_DUPLEX, 0.5, (255, 255, 255))

            # Vẽ các landmarks
            if(FIX_AFTER_MODEL):
                x1, y1, x2, y2 =b[0:4]
                b[5],b[6] = find_darkest_point_in_box_center(img_raw,b[5],b[6],((x2-x1)/4),((y2-y1))/8)
                b[7],b[8] = find_darkest_point_in_box_center(img_raw,b[7],b[8],((x2-x1)/4),((y2-y1))/8)

            cv2.circle(img_raw, (b[5], b[6]), 1, (0, 0, 255), 4)
            cv2.circle(img_raw, (b[7], b[8]), 1, (0, 255, 255), 4)
            cv2.circle(img_raw, (b[9], b[10]), 1, (255, 0, 255), 4)
            cv2.circle(img_raw, (b[11], b[12]), 1, (0, 255, 0), 4)
            cv2.circle(img_raw, (b[13], b[14]), 1, (255, 0, 0), 4)

        # Hiển thị khung hình
        cv2.imshow('Webcam Detection', img_raw)
    
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Chạy hàm phát hiện từ webcam
if True: detect_from_webcam()


AttributeError: module 'cv2' has no attribute 'VideoCapture'

In [16]:
eye_points = np.array([[100, 150], [200, 120]])  # Demo vị trí hai mắt
eye_dist = np.linalg.norm(eye_points[1] - eye_points[0])
print(eye_dist)

104.4030650891055


In [2]:
import cv2
import numpy as np

# Đọc hình ảnh kính (nền trong suốt phải là PNG)
glasses_img = cv2.imread("/home/four/Downloads/matkinh (1).png", cv2.IMREAD_UNCHANGED)
height_glasses_img, width_glasses_img = glasses_img.shape[:2]
new_height = 3 * height_glasses_img
expanded_img = np.zeros((new_height, width_glasses_img, 4), dtype=np.uint8)
start_y = (new_height - height_glasses_img) // 2
expanded_img[start_y:start_y + height_glasses_img, :, :] = glasses_img
glasses_img = expanded_img
glass_eye_points = np.array([[0.25, 0.4], [0.75, 0.4]])  # normalized
width_glasses_img, height_glasses_img = glasses_img.shape[1], glasses_img.shape[0]
width_eye_glass = int(0.5 * width_glasses_img)




def overlay_glasses(frame, eye_points, glasses_img, glass_eye_points):
    eye_dist = np.linalg.norm(eye_points[1] - eye_points[0])
    glass_width = int(glasses_img.shape[1] * (eye_dist / width_eye_glass))
    glass_height = int(glasses_img.shape[0] * (glass_width / glasses_img.shape[1]))
    resized_glasses = cv2.resize(glasses_img, (glass_width, glass_height), interpolation=cv2.INTER_AREA)

    dx, dy = eye_points[1][0] - eye_points[0][0], eye_points[1][1] - eye_points[0][1]
    angle = -np.degrees(np.arctan2(dy, dx))
    center = (glass_width // 2, glass_height // 2)
    rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1)

    rotated_glasses = cv2.warpAffine(
        resized_glasses, 
        rotation_matrix, 
        (glass_width, glass_height), 
        flags=cv2.INTER_LINEAR, 
        borderMode=cv2.BORDER_CONSTANT, 
        borderValue=(0, 0, 0, 0)
    )

    glasses_center = np.mean(eye_points, axis=0).astype(int)
    top_left = (glasses_center[0] - glass_width // 2, glasses_center[1] - glass_height // 2)
    x1, y1 = max(0, top_left[0]), max(0, top_left[1])
    x2, y2 = min(frame.shape[1], x1 + glass_width), min(frame.shape[0], y1 + glass_height)

    overlay = rotated_glasses[:y2 - y1, :x2 - x1]
    alpha = overlay[:, :, 3:4] / 255.0

    frame[y1:y2, x1:x2] = (
        alpha * overlay[:, :, :3] + (1 - alpha) * frame[y1:y2, x1:x2]
    ).astype(np.uint8)

    return frame


cap = cv2.VideoCapture(0)  # Mở webcam

while cap.isOpened():
    ret, img_raw = cap.read()
    if not ret:
        break

    data_faces = detect_faces(img_raw)  # Giả sử hàm detect_faces đã trả về thông tin các gương mặt
    for b in data_faces:
        # Lấy vị trí mắt của từng gương mặt
        eye_points = np.array([[b[5], b[6]], [b[7], b[8]]])  # Demo vị trí hai mắt
        # Áp dụng overlay kính cho từng gương mặt
        img_raw = overlay_glasses(img_raw, eye_points, glasses_img, glass_eye_points)

    cv2.imshow('WebCam with Glasses', img_raw)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


NameError: name 'detect_faces' is not defined